In [ ]:
import polars as pl
import deltalake
import os
import sys

In [ ]:
pl.__version__

In [ ]:
df1 = pl.DataFrame(
    {
        "a": ["A", "B", "C"],
        "b": [4, 5, 6],
    }
)
df1

In [ ]:
df1.write_delta("data/delta-table")


In [ ]:
location = "/Users/tweber/Gits/depictio/minio_data/depictio-bucket/"
# location = f"../../{location}"
for root, dirs, files in os.walk(location):
    for file in files:
        print(file)
        print(os.path.join(root, file))
        if (file.endswith(".parquet")) and ("test_table" not in os.path.join(root, file)):
            print("parquet")
            delta_table = "/".join(os.path.join(root, file).split("/")[:-1]) + "/"
print(delta_table)
delta_table = "/Users/tweber/Gits/depictio/minio_data/depictio-bucket/65e5f007bad32df857a53cf2/65ef2b8624e251dd1fb14e9c/65ef2b8624e251dd1fb14e9d/"

In [ ]:
!tree -h -D $location

In [ ]:
!tree -h -D $delta_table

In [ ]:
with pl.Config(fmt_str_lengths=50):
    print(pl.read_delta(delta_table, version=0).select("depictio_aggregation_time"))

In [ ]:
!ls -alh ../../minio_data/depictio-bucket/65e5f007bad32df857a53cf2/65ef057b6832d5ac36e3a1df/65ef057b6832d5ac36e3a1e0/_delta_log

In [ ]:
!ls ../.././minio_data/depictio-bucket/65e5f007bad32df857a53cf2/65ef0cf6ba771d68d82abe9e/65ef0cf6ba771d68d82abe9f

In [ ]:
dt = deltalake.DeltaTable("../../minio_data/depictio-bucket/65e5f007bad32df857a53cf2/65ef0cf6ba771d68d82abe9e/65ef0cf6ba771d68d82abe9f/")

In [ ]:
import pandas as pd


df = pd.DataFrame([{"id": 1, "name": "foo"}, {"id": 2, "name": "bar"}, {"id": 3, "name": "baz"}])

pl_df = pl.DataFrame(df)

pl_df.write_delta("../../minio_data/depictio-bucket/test_table/", mode="overwrite")

In [ ]:
pl.read_delta("../../minio_data/depictio-bucket/test_table", version=0)

In [ ]:
pl.read_delta("../.././minio_data/depictio-bucket/65e5f007bad32df857a53cf2/65ef0ef8cbb1b4b72250fb0f/65ef0ef8cbb1b4b72250fb10/")

In [ ]:
pl.__version__

In [ ]:
# deltalake version
import deltalake
import pandas as pd

df = pd.DataFrame([{"id": 1, "name": "foo"}, {"id": 2, "name": "bar"}, {"id": 3, "name": "baz"}])
deltalake.write_deltalake("./data/delta-table", df, mode="overwrite")

# Load the table
table = deltalake.DeltaTable("./data/delta-table")
table.to_pandas()

In [ ]:
import boto3

s3 = boto3.client("s3", endpoint_url="http://localhost:9000", aws_access_key_id="minio", aws_secret_access_key="minio123", region_name="us-east-1")
from pprint import pprint

pprint(s3.list_buckets())

In [ ]:
import deltalake
import pandas as pd

# DataFrame creation
df = pd.DataFrame([{"id": 1, "name": "foo"}, {"id": 2, "name": "bar"}, {"id": 3, "name": "baz"}])

# MinIO storage options
storage_options = {
    "endpoint_url": "http://localhost:9000",  # Change localhost to your MinIO server IP if necessary
    "aws_access_key_id": "minio",
    "aws_secret_access_key": "minio123",
    "use_ssl": "false",
    "AWS_REGION": "us-east-1",  # Example region, adjust as needed
    "signature_version": "s3v4",
    "AWS_ALLOW_HTTP" : "true",
    "AWS_S3_ALLOW_UNSAFE_RENAME": "true"
}

# Write to MinIO
deltalake.write_deltalake("s3://depictio-bucket/data/delta-table4", df, storage_options=storage_options, mode="overwrite")

# # Load the table from MinIO
# table = deltalake.DeltaTable("s3://your-bucket-name/data/delta-table", storage_options=storage_options)
# df_from_minio = table.to_pandas()

In [ ]:
table = deltalake.DeltaTable("s3://depictio-bucket/data/delta-table4", storage_options=storage_options)
df_from_minio = table.to_pandas()
df_from_minio

In [ ]:
# do the same with polars
import polars as pl


df = pd.DataFrame([{"id": 1, "name": "foo"}, {"id": 2, "name": "bar"}, {"id": 3, "name": "baz"}, {"id": 4, "name": "baz"}])
df = pl.DataFrame(df)
df.write_delta("s3://depictio-bucket/data/delta-table5", storage_options=storage_options, mode="overwrite")
df

In [ ]:
import boto3
from botocore.exceptions import NoCredentialsError

# MinIO credentials
minio_endpoint = "http://localhost:9000"  # Change to your MinIO server URL
access_key = "minio"
secret_key = "minio123"
bucket_name = "depictio-bucket"
destination_prefix = "67a25c82fea6466823de362"  # Remove 's3://' when using boto3

storage_options = {
    "endpoint_url": minio_endpoint,
    "aws_access_key_id": access_key,
    "aws_secret_access_key": secret_key,
    "use_ssl": "false",
    "AWS_REGION": "us-east-1",
    "signature_version": "s3v4",
    "AWS_ALLOW_HTTP": "true",
    "AWS_S3_ALLOW_UNSAFE_RENAME": "true"
}

import deltalake.exceptions
import polars as pl
import deltalake

try:
    df = pl.read_delta(f"s3://{bucket_name}/{destination_prefix}", storage_options=storage_options)
except deltalake.exceptions.TableNotFoundError as e:
    print("Table not found:", e)
    # Handle the error or provide a fallback
    # For example, you could create a new Delta table or log the error


In [ ]:
# read using polars from minio
# list versions of the table in the delta table

import polars as pl
storage_options = {
    "endpoint_url": "http://localhost:9000",  # Change localhost to your MinIO server IP if necessary
    "aws_access_key_id": "minio",
    "aws_secret_access_key": "minio123",
    "use_ssl": "false",
    "AWS_REGION": "us-east-1",  # Example region, adjust as needed
    "signature_version": "s3v4",
    "AWS_ALLOW_HTTP" : "true",
    "AWS_S3_ALLOW_UNSAFE_RENAME": "true"
}
df = pl.scan_delta("s3://depictio-bucket/662acab447d998cd889678c1/6647d03b57d1fae8d89dce8d/6647d03b57d1fae8d89dce8e/", storage_options=storage_options).collect()
df


In [ ]:
pl.__version__

In [ ]:
type(df["cell"].unique())

In [ ]:
list(df["cell"].unique())

In [ ]:
import polars as pl

# Function to add filter criteria to a list
def add_filter(filter_list, interactive_component_type, column_name, value, min_value=None, max_value=None):
    print(filter_list)
    if interactive_component_type in ["Select", "MultiSelect", "SegmentedControl"]:
        if value:
            filter_list.append(pl.col(column_name).is_in(value))
    
    elif interactive_component_type == "TextInput":
        if value:
            filter_list.append(pl.col(column_name).str.contains(value))
    
    elif interactive_component_type == "Slider":
        if value:
            filter_list.append(pl.col(column_name) == value)
    
    elif interactive_component_type == "RangeSlider":
        if value:
            filter_list.append((pl.col(column_name) >= value[0]) & (pl.col(column_name) <= value[1]))

# Function to apply the cumulative filter to the DataFrame
def apply_filters(df, filter_list):
    if filter_list:
        combined_filter = filter_list[0]
        for filt in filter_list[1:]:
            combined_filter &= filt
        return df.filter(combined_filter)
    return df

# Example usage
filter_list = []

# Example interactive components and values
interactive_components = [
    {"type": "Select", "column": "category", "value": ["A", "B"]},
    {"type": "TextInput", "column": "name", "value": "John"},
    {"type": "Slider", "column": "age", "value": 30},
    {"type": "RangeSlider", "column": "salary", "value": [50000, 100000]},
]

# Populate the filter list
for component in interactive_components:
    add_filter(
        filter_list,
        interactive_component_type=component["type"],
        column_name=component["column"],
        value=component["value"]
    )

# Sample DataFrame
df = pl.DataFrame({
    "category": ["A", "B", "C", "A"],
    "name": ["John", "Jane", "John", "Doe"],
    "age": [30, 25, 40, 30],
    "salary": [60000, 80000, 70000, 90000]
})

# Apply the filters
filtered_df = apply_filters(df, filter_list)

print(filtered_df)